# 让大模型自己生成代码并测试

# 生成与迭代反馈

In [41]:
import os
import json
from pprint import pprint
import subprocess
import json
import argparse
import numpy as np
import re
from scipy.fftpack import fft
from typing import Tuple
from pathlib import Path
from typing import Dict
from langchain_openai import ChatOpenAI
from functools import partial
from scipy.signal import lfilter

# --------------------------------------- API配置 ----------------------------------------
os.environ["OPENAI_API_KEY"] = "sk-cwiXW4bSS8W4ykhB8Z2K9W1fGnHMiNcF6qoFMIqpAK5Qw6Zs"
os.environ["OPENAI_API_BASE"] = "https://api.closeai-proxy.xyz/v1"
model = ChatOpenAI(model="gpt-4o")

# 流式输出结果
def stream_and_model(question, _model) -> str:
    all_text = ""
    for chunk in _model.stream(question):
        # print(chunk.content, end="", flush=True)
        all_text += chunk.content
    return all_text
stream = partial(stream_and_model, _model=model)
# --------------------------------------- 基础配置 ----------------------------------------
task_name = "fft4_(8guide4)" #fft4 fft8 fft4_(8guide4) fir16
workspace = Path("Result")
questions_path = Path("G:/Desktop/LLM_ICDesign/课题") / task_name
problem_prompt_path = questions_path / f"{task_name}_prompt.txt"

# ----------------------------------- LLM 第一次生成结果 -----------------------------------
history = []
history.append("System: You are a Verilog expert, I need your help to write verilog code for me")
history.append(f"User: {problem_prompt_path.read_text()}")
history.append(f"User: above is all the information for you to solve the problem. Remember, your code's top module name should always be `TopModule`")
history.append(f"User: Now you analyze the problem and at last write the code for me.")

ans = stream(str(history))
history.append(f"AI: {ans}")

def remove_markdown(json_text) -> str:
    json_text = json_text.replace("```json", "").replace("```", "")
    return json_text

def abstract_code_from_answer(ans: str) -> Dict:
    prompt = f"""User: You have analyzed the verilog code writing problem and write the code for me.
I need you to extract the analysis and code from the above text.
You should output in Json format as below:
{{
    analysis: "Your analysis here",
    code: "Your code here"
}}
-------------------------
Here is the text:
{ans}
"""
    ans = stream(prompt)
    return json.loads(remove_markdown(ans))

code = abstract_code_from_answer(ans)["code"]
print("\n-----------------------------code generate by LLM -------------------------------------------")
print(code)
history.append("AI: The first generated code:")
history.append(code)

# -----------------------------------对LLM 生成的结果进行测试-----------------------------------    
# ----------------------------------- 声明语法和功能检查工具 -----------------------------------   
def read_coefficients(input_file):
    with open(input_file, 'r') as file:
        coef = []
        for line in file:
            # 去除行末的换行符和空格
            line = line.strip()
            # 处理每一行的数据
            if line:
                coef.append(int(line))
    return coef

def fir_eval(h_coe=None,in_bitwidths = 16,out_bitwidths = 32,VCD_dir=None,signinter=False):

    in_name = "xin"
    out_name = "yout"
    signals, timestamps=read_vcd(VCD_dir)

    check_port_name = []
    for signal_id, signal in signals.items():
        check_port_name.append(signal['name'])
    
    assert in_name in check_port_name ,"Make sure your input port name is xin!"
    assert out_name in check_port_name ,"Make sure your output port name is yout!"  
    assert 'clk' in check_port_name,"Make sure your clock name is clk!"
    assert 'xin_en' in check_port_name,"Make sure you have xin_en signal!"      
    assert 'yout_valid'in check_port_name,"Make sure you have yout_valid signal!"    

    valid_time = 0
    en_time = 0
    #数据准备for
    for signal_id, signal in signals.items():
        # #提取clk时钟
        if signal['name'] == 'clk':
            clk_period = signal['changes'][4][0]-signal['changes'][2][0]
        
        #提取en值为高的时刻
        if signal['name'] == "xin_en":
            for change in signal['changes']:
                time, value = change
                if(value == '1'):
                    if en_time<int(time):
                        en_time = int(time)        

        #提取valid值为高的时刻
        if signal['name'] == 'yout_valid':
            for change in signal['changes']:
                time, value = change
                if(value == '1'):
                    if valid_time<int(time):
                        valid_time = int(time)

    in_time_last = en_time
    out_time_last = valid_time
    for signal_id, signal in signals.items(): 

        if signal['name'] == in_name:
            spilt_in=[]
            #根据Valid统计除X值以外有效y的个数
            for change in signal['changes']:
                time , value = change
                if(int(time) >= en_time):
                    if int(time) - in_time_last <= clk_period:
                        spilt_in.append(value)
                        in_time_last = int(time)
                    else:
                        #补齐相同的输入
                        for i in range(int((int(time)-in_time_last)/clk_period)-1):
                            spilt_in.append(spilt_in[-1])
                        spilt_in.append(value)
                        in_time_last = int(time)
            xin_len= len(spilt_in)        

        if signal['name'] == out_name:
            spilt_out=[]
            #根据Valid统计除X值以外有效y的个数
            for change in signal['changes']:
                time , value = change
                if(int(time) > valid_time):
                    if int(time) - out_time_last <= clk_period:
                        spilt_out.append(value)
                        out_time_last = int(time)
                    else:
                        #补齐相同的输出
                        for i in range(int((int(time)-out_time_last)/clk_period)-1):
                            spilt_out.append(spilt_out[-1])
                        spilt_out.append(value)
                        out_time_last = int(time)
            yout_len= len(spilt_out)
            
    if(xin_len > yout_len):
        xin_len = yout_len
    assert xin_len==yout_len,"The amout of Input  and Output is different!"

    in_array = np.empty(xin_len)
    out_array = np.empty(yout_len)

    for signal_id, signal in signals.items():     
        # 寻找输入值
        if signal['name'] == in_name:
            for i in range(xin_len):
                value = spilt_in[i]
                if(signinter):
                    in_array[i]=signed_bin_to_dec(value,in_bitwidths)
                else:
                    in_array[i]=int(value,2)
            
        # 寻找输出结果
        if signal['name'] == out_name:
            #循环提取
            for i in range(yout_len):
                value = spilt_out[i]
                if signinter :
                    out_array[i]=signed_bin_to_dec(value,out_bitwidths)
                else:
                    out_array[i]=int(value,2)

    #FIR滤波器计算参考结果
    filtered_signal = lfilter(h_coe, 1.0, in_array)
    out_array = np.array(out_array)
    mse_error = complex_MSE(filtered_signal,out_array)
    

    print("test numbers:",xin_len)
    print("MSE result:",mse_error) 
    return mse_error

def arg_manage():

    parser = argparse.ArgumentParser()
    parser.add_argument("--fft_channels" ,type=int,required=True,help="FFT 总通道数")
    parser.add_argument("--in_bitwidths" ,type=int,required=True,help="单个数据输入位宽")
    parser.add_argument("--out_bitwidths",type=int,required=True,help="单个数据输出位宽")
    parser.add_argument("--VCD_dir",type=str,required=True,help="VCD文件路径")
    args = parser.parse_args()
    return args
    # 解析命令行参数

def read_vcd(filename):
    signals = {}
    timestamps = []
    current_time = 0
    
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            
            # 解析信号声明
            if line.startswith('$var'):
                parts = line.split()
                signal_type = parts[1]    # 例如 wire, reg 等
                signal_id = parts[3]      # 唯一标识符
                signal_name = parts[4]    # 信号名称
                signals[signal_id] = {'name': signal_name, 'type': signal_type, 'changes': []}
            
            # 解析时间戳
            elif line.startswith('#'):
                current_time = int(line[1:])
                timestamps.append(current_time)
            
            # 解析信号值变化 (二进制值变化)
            elif re.match(r'^[01]', line):
                value = line[0]
                signal_id = line[1:]
                if signal_id in signals:
                    signals[signal_id]['changes'].append((current_time, value))
            
            # 解析信号值变化 (多位二进制或十六进制)
            elif re.match(r'^[b|B|h|H]', line):
                parts = line.split()
                value = parts[0][1:]  # 获取值 (去掉前导字符)
                signal_id = parts[1]
                if signal_id in signals:
                    signals[signal_id]['changes'].append((current_time, value))

    return signals, timestamps

def print_waveform_details(signals):
    print("Waveform Details:\n")
    for signal_id, signal in signals.items():
        print(f"Signal ID: {signal_id}")
        print(f"  Name: {signal['name']}")
        print(f"  Type: {signal['type']}")
        print("  Changes:")
        for change in signal['changes']:
            time, value = change
            print(f"    Time: {time}, Value: {value}")
        print(signal['changes'][0])
        print("")

def signed_bin_to_dec(bin_str,bitwidths):
    
    # 将二进制字符串转换为整数
    value = int(bin_str, 2)
    
    if value & (1 << (bitwidths - 1)):
        value -= (1 << bitwidths)
    
    return value

def complex_MSE(x,x_ref):
    return np.mean(np.abs(x-x_ref)**2)

def fft_eval(fft_channels=8,in_bitwidths=24,out_bitwidths=24,VCD_dir=None):
    
    in_real_names  = []
    in_image_names = []
    out_real_names  = []
    out_image_names = []

    for i in range(fft_channels):
        in_real_names.append("x"+str(i)+"_real")
        in_image_names.append("x"+str(i)+"_imag")

    for i in range(fft_channels):
        out_real_names.append("y"+str(i)+"_real")
        out_image_names.append("y"+str(i)+"_imag")

    signals, timestamps = read_vcd(VCD_dir)
    
    check_port_name = []
    for signal_id, signal in signals.items():
        check_port_name.append(signal['name'])

    assert all(item in check_port_name for item in in_real_names) & all(item in check_port_name for item in in_image_names),"Make sure your input port name is x{$channel_index}_real or x{$channel_index}_imag!"
    assert all(item in check_port_name for item in out_real_names) & all(item in check_port_name for item in out_image_names),"Make sure your output port name is y{$channel_index}_real or y{$channel_index}_imag!"
    assert 'yout_valid' in check_port_name,"Make sure you have yout_valid signal!"  
    valid_time = 0
    in_num = np.empty((fft_channels))
    out_num = np.empty((fft_channels))
    
    #数据准备for
    for signal_id, signal in signals.items():
        #提取valid值为高的时刻
        if signal['name'] == 'yout_valid':
            for change in signal['changes']:
                time, value = change
                if(value == '1'):
                    if valid_time<int(time):
                        valid_time = int(time)
                    
    for signal_id, signal in signals.items():        
        #提取输入数量
        if signal['name'] in in_real_names:
            in_real_length = len(signal['changes'])
            index_in_real = in_real_names.index(signal['name'])
            in_num[index_in_real] = in_real_length
        
        if signal['name'] in out_real_names:
            index_out_real = out_real_names.index(signal['name'])
            spilt_signals=[]
            #根据Valid统计除X值以外有效y的个数
            for change in signal['changes']:
                time , value = change
                if(int(time) >= valid_time):
                    spilt_signals.append(value)
            out_num[index_out_real] = len(spilt_signals)
    
    assert np.all(in_num==in_num[0]), "Input channels have different lengths!"
    assert np.all(out_num==out_num[0]), "Output channels have different lengths!"

    #FFT电路输入输出有效组数
    eval_in_groups = int(in_num[0])
    eval_out_groups = int(out_num[0])

    if eval_in_groups > eval_out_groups:
        eval_in_groups = eval_out_groups

    assert eval_in_groups==eval_out_groups,"Input channels and Output channels have different lengths!"

    in_array = np.empty((fft_channels,eval_in_groups,2))
    out_array = np.empty((fft_channels,eval_out_groups,2))

    for signal_id, signal in signals.items():
        # 寻找各输入通道实数输入结果
        if signal['name'] in in_real_names:
            index_in_real = in_real_names.index(signal['name'])
            
            for i in range(eval_in_groups):
                time , value = signal['changes'][i]
                in_array[index_in_real][i][0] = signed_bin_to_dec(value,in_bitwidths)
                
        # 寻找各输入通道虚数输入结果
        if signal['name'] in in_image_names:
            index_in_image = in_image_names.index(signal['name'])
   
            for i in range(eval_in_groups):
                time , value = signal['changes'][i]
                in_array[index_in_image][i][1] = signed_bin_to_dec(value,in_bitwidths)

        
        # 寻找各输出通道实数输出结果
        if signal['name'] in out_real_names:
            index_out_real = out_real_names.index(signal['name'])
            
            #去除X值
            spilt_signals=[]
            for change in signal['changes']:
                time , value = change
                if(int(time) >= valid_time):
                    spilt_signals.append(change)
            #循环提取
            for i in range(eval_out_groups):
                time , value = spilt_signals[i]
                out_array[index_out_real][i][0] = signed_bin_to_dec(value,out_bitwidths)
        
        #寻找各输出通道虚数输出结果
        if signal['name'] in out_image_names:
            index_out_image = out_image_names.index(signal['name'])
            
            #去除X值
            spilt_signals=[]
            for change in signal['changes']:
                time , value = change
                if(int(time) >= valid_time):
                    spilt_signals.append(change)
            #循环提取
            for i in range(eval_out_groups):
                time , value = spilt_signals[i]
                out_array[index_out_image][i][1] = signed_bin_to_dec(value,out_bitwidths)
    
    #合成复数
    in_complex = np.transpose(in_array[:,:,0] + 1j * in_array[:,:,1])
   
    fft_ref = fft(in_complex)  
    # print("FFT result:", fft_ref)
    
    #合成复数
    out_complex = np.transpose(out_array[:,:,0] + 1j * out_array[:,:,1])
    # print("RTL calculate result:",out_complex)
    
    mse_error = complex_MSE(out_complex,fft_ref)
    print("Each channel total test input numembers:",eval_in_groups)
    print("MSE result:",mse_error)
    if mse_error is None:
        print("Warning: mse_error is None")
    return mse_error

def solve_syntax_error(code: str, syntax_err: str) -> Tuple[str, str]:
    """return analyze and corrected code"""
    prompt = f"""Please solve the syntax error according to the error message below:
======================Code======================
{code}
======================Error======================
{syntax_err}
=================================================
Please analysis the error, and at last give me the corrected code.
"""
    # Syntax Error 的改进方向：
    ans = stream(prompt)
    ans = abstract_code_from_answer(ans)
        
    analysis = ans["analysis"]
    corrected_code = ans["code"]
    
    corrected_code = corrected_code.replace("```verilog", "").replace("```", "")
    return analysis, corrected_code
    
def solve_functional_error(code: str, functional_error: str) -> Tuple[str, str]:
    """return analyze and corrected code"""
    prompt = f"""Please solve the functional error according to the error message below:
======================Code======================
{code}
======================Error======================
{functional_error}
=================================================
Please analysis the error, and at last give me the corrected code.
"""
    # Functional Error 的改进较为复杂，不同的电路会有不同的debug方法，同学们可以挑选一种电路，尝试为他设计一种改进方法。
    # 如真值表电路，可以添加一条思维链，提取代码中的所有公式，然后与真值表对比是否正确，或者用大模型将代码转化为python代码，用真值表的结论来演算，获得更多反馈来给大模型目前的结果打分
    
    ans = stream(prompt)
    ans = abstract_code_from_answer(ans)
        
    analysis = ans["analysis"]
    corrected_code = ans["code"]
    
    corrected_code = corrected_code.replace("```verilog", "").replace("```", "")
    return analysis, corrected_code

def is_their_any_functional_error(vvp_output: str) -> bool:
    prompt = f"""Here is the output runned by IVerilog.
    Please check if it means there is any functional error in the code.
    =========================
    {vvp_output}
    =========================
    If there is any functional error, the text above will tell you there is mismatch in some Output.
    If all the output has no mismatch, then there is no functional error.
    Give me your analysis and at the end firmly tell me if there is any functional error or not.
    """
    ans = stream(prompt)
    
    prompt = f"""Please tell me if there is any functional error in the code. You will output in Json format as below:
    if there is no functional_error, functional_ok should be True, otherwise False
    {{
        analysis: "Your analysis here",
        functional_ok: bool
    }}
    =============================
    Here is the text: {ans}
    """
    ans = stream(prompt)
    return json.loads(remove_markdown(ans))["functional_ok"]

def is_mse_error_ok(task_name) -> bool:
    # 这里来通过与标准的ref电路的输出结果计算MSE来判断 ，如果MSE的结果大于一定阈值，那么就说明还是有功能错误
    if task_name == "fir16":
        VCD_dir =  workspace / task_name / "Result.vcd"
        coef_dir =  workspace / task_name / "fir_coef.txt"
        coef = read_coefficients(coef_dir)
        mse_error = fir_eval(h_coe=coef,VCD_dir = VCD_dir)
    else:
        task_params = {
                "fft4": {
                    "fft_channels": 4,
                    "in_bitwidths": 8,
                    "out_bitwidths": 10
                },
                "fft4_(8guide4)": {
                    "fft_channels": 4,
                    "in_bitwidths": 24,
                    "out_bitwidths": 24
                },
                "fft8": {
                    "fft_channels": 8,
                    "in_bitwidths": 24,
                    "out_bitwidths": 24
                }
                
            }
        params = task_params.get(task_name, {})
        fft_channels = params.get("fft_channels")
        in_bitwidths = params.get("in_bitwidths")
        out_bitwidths = params.get("out_bitwidths")
        VCD_dir =  workspace / task_name / "Result.vcd"
        mse_error =fft_eval(fft_channels=fft_channels, in_bitwidths=in_bitwidths, out_bitwidths=out_bitwidths, VCD_dir=VCD_dir)
    if mse_error > 5:
        return False
    else:
        return True

def run_iverilog(task_name: str, rtl_code: str, tb_code: str,  subref_code: str = "") -> Tuple[str, str]:
    """编译 Verilog 代码

    Returns:
        Tuple[str, str]: syntax, functional
    """
    rtl_code = rtl_code.replace("```verilog", "").replace("```", "")
    tb_code = tb_code.replace("```verilog", "").replace("```", "")
    # ref_code = ref_code.replace("```verilog", "").replace("```", "")
    
    workdir = workspace / task_name
    workdir.mkdir(parents=True, exist_ok=True) # 如果目录不存在，则会创建它及其父目录
    
    rtl_path = workdir / f"{task_name}_solve.v"
    rtl_path.write_text(rtl_code)
    
    tb_path = workdir / f"{task_name}_test.v"
    tb_path.write_text(tb_code)
    
    # ref_path = workdir / f"{task_name}_ref.v"
    # ref_path.write_text(ref_code)
    
    if subref_code:
        subref_path = workdir / "subref.v"
        subref_path.write_text(subref_code)
    
    output_file_path = workdir / "result.out"  # 定义了编译后的输出文件路径
    
    # 使用 iverilog 工具编译 solve.v、test.v 和 ref.v 文件，并将输出写入到 result.out 文件中
    compile_cmd = ["iverilog", "-o", str(output_file_path), "-I", str(workdir), str(rtl_path), str(tb_path)]
    if subref_code:
        compile_cmd.append(str(subref_path))
    
    result = subprocess.run(compile_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    syntax_err = result.stderr.decode("utf-8")
    
    if syntax_err:
        return syntax_err, None , None
    
    # 运行仿真并生成 VCD 文件
    vcd_file_path = workdir / "Result.vcd"
    result = subprocess.run(["vvp", str(output_file_path)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    functional_error = result.stdout.decode("utf-8")
    
    return None, functional_error, str(vcd_file_path)
# ----------------判断 是否具有语法或者功能错误 以及在没有错误的时候检查计算是否正确----------------
# 原始的tb code 、 ref code 、subref code 路径  便于后续的调用
tb_code = (questions_path / f"{task_name}_tb.v").read_text().replace("RefModule", "TopModule")
# ref_code = (questions_path / f"{task_name}_ref.v").read_text()
subref_file = questions_path / "subref.v"
if subref_file.exists():
    subref_code = subref_file.read_text()
else:
    subref_code = ""
    
# 一直循环直到没有语法和功能错误 或者达到最大迭代次数
max_iter = 5
current_round = 1

# # ----------------- Test -----------------
# rtl_code = code
# syntax_err, functional_error, vcd_file = run_iverilog(task_name, rtl_code, tb_code,subref_code)
# print("sunatax error:",syntax_err)
# if syntax_err != None:
#     print("over")
# else:
#     print("functional error:",functional_error)
#     mse_ok = is_mse_error_ok(task_name)

# -------------------------------------------------- 迭代生成 --------------------------------------------------
while True:
    print(f"====================================================DEBUG ROUND {current_round}, current history length: {len(str(history))}=========================================================")
    # 使用iverilog来检查是否有语法错误和功能错误
    # 获取上一次生成的code的代码
    rtl_code = code
    syntax_err, functional_error, vcd_file = run_iverilog(task_name, rtl_code, tb_code,subref_code)
    # 判断是否有语法错误
    if syntax_err:
        print("Syntax error:", syntax_err)
        history.append("Tool: Syntax error occured! Tool output is below:")
        history.append(syntax_err)
        analysis, code = solve_syntax_error(code, syntax_err)
    else:
        print("No Syntax error")
        functional_ok = is_their_any_functional_error(functional_error)
        # 判断是否有功能错误
        if not functional_ok :
            print("Functional error:", functional_error)
            history.append("Tool: Functional error occured! Tool output is below")
            history.append(functional_error)
            analysis, code = solve_functional_error(code, functional_error)
        else:
            print("No Functional error")
        # 与正确结果对比
        # if not syntax_err and not functional_ok:
            mse_ok = is_mse_error_ok(task_name)
            if not mse_ok:
                print("MSE is large")
                history.append("Tool: Functional error occured! Tool output is below")
                history.append("The MSE of the result is larger than the threadhold, which means the calculation steps have promblems.Please check the code again!")
                history.append("You should pay close attention to the code in the middle of the generate statement in the source code. Other code does not need to be modified, but you should also pay attention to the range of m k should match the size dimensions of en_comnect, xm_deal, and xm_imag arrays")
                analysis, code = solve_functional_error(code, functional_error)
            else:
                print("Congratulations! All is well!")
                # 将代码写入文件
                llm_gen_path = workspace / task_name / f"{task_name}_llm_gen.v"
                with llm_gen_path.open("w") as f:
                    f.write(code)
                    print(f"Code has been written to {llm_gen_path}")
                break
            
    if current_round > max_iter:
        print("Max Iteration reached")
        break
    else :
        current_round += 1
        
    history.append(f"AI: Analysis for the error: {analysis}")
    history.append(f"AI: modified code:")
    history.append(code)


-----------------------------code generate by LLM -------------------------------------------
`timescale 1ns/100ps
module TopModule (
       input                    clk,
       input                    rstn,
       input                    en,

       input signed [23:0]      x0_real,
       input signed [23:0]      x0_imag,
       input signed [23:0]      x1_real,
       input signed [23:0]      x1_imag,
       input signed [23:0]      x2_real,
       input signed [23:0]      x2_imag,
       input signed [23:0]      x3_real,
       input signed [23:0]      x3_imag,

       output                   yout_valid,
       output signed [23:0]     y0_real,
       output signed [23:0]     y0_imag,
       output signed [23:0]     y1_real,
       output signed [23:0]     y1_imag,
       output signed [23:0]     y2_real,
       output signed [23:0]     y2_imag,
       output signed [23:0]     y3_real,
       output signed [23:0]     y3_imag
       );

   // Operating data
   wire signed [23:0]